In [ ]:
import torch
from torch_geometric.data import Data



In [ ]:
from utils.path_manage import get_files

data, lookup, ASD_dictionary, BCE_dictionary = get_files()


In [ ]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())



In [ ]:
data.keys

In [ ]:
data.edge_index